In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


plt.rcParams.update({'figure.figsize':(15,6)})
plt.rcParams.update({'font.size':18})

In [2]:
df = pd.read_csv('feature_engineered_data.csv')
df.head(3)

,Gender_Male,Ever_Married_Yes,Graduated_Yes,Profession_Doctor,Profession_Engineer,Profession_Entertainment,Profession_Executive,Profession_Healthcare,Profession_Homemaker,Profession_Lawyer,...,Profession_unknown_Spending_Score_High,Profession_unknown_Spending_Score_Low,Work_Experience_Entry_Spending_Score_High,Work_Experience_Entry_Spending_Score_Low,Work_Experience_High_Spending_Score_High,Work_Experience_High_Spending_Score_Low,Work_Experience_Intermediate_Spending_Score_High,Work_Experience_Intermediate_Spending_Score_Low,Work_Experience_unknown_Spending_Score_High,Work_Experience_unknown_Spending_Score_Low
0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [3]:
df.shape

(7902, 134)

In [8]:
data = df.drop(columns=['Segmentation'])
y = df[['Segmentation']]

In [4]:
# function that collects all combination of the features
def rSubset(arr, min_col, max_col):
    l = []
    for r in range(min_col,max_col):
        l.append(list(combinations(arr, r)))
    return l

In [ ]:
def fs_linear_reg_3(col, col_y):
    # この48を自動で、最新月を除外するようにしないといけない！
    m = len(data)

    y_t = data[[col_y]][:m]
    X_t = data[col][:m]

    X_train, X_test, y_train, y_test = X_t[:m-6], X_t[m-6:], y_t[:m-6], y_t[m-6:]
    reg = LinearRegression().fit(X_train, y_train)
    y_pred = reg.predict(X_test)

    y_te = y_t.copy()
    y_te['current_amount'] = data.current_amount
    y_te.index = data.month[:len(y_te)]
    y_te['predict'] = reg.predict(X_t).tolist()
    y_te['predict'] = y_te['predict'].apply(lambda x: x[0])
    y_te['diff'] = round(y_te['predict'] - y_te[col_y],2)
    y_te['diff'] = y_te['diff'].apply(lambda x: "{:,}".format(x))

    y_te['month_m'] = y_te.index.month
    y_te = y_te.merge(month_coef, on = 'month_m', how = 'left')
    y_te.index = data.month[:len(y_te)]
    y_te = y_te.drop(columns=['month_m'])
    y_te['new_predict'] = y_te.predict * y_te.coef

    y = y_te['current_amount']
    y_p = y_te['new_predict']
    output1 = round(mean_squared_error(y, y_p),4)

    y_pr = y_te['current_amount'][m-6:]
    y_p_pr = y_te['new_predict'][m-6:]
    output2 = round(mean_squared_error(y_pr, y_p_pr),4)

    return output1, output2

In [ ]:
col_l = []
overall_l = []
pred_l = []
for r in range(4):
    for i in range(len(rSubset(cols,3,7)[r])):
        col_l.append(list(rSubset(cols,3,7)[r][i]))
        overall_l.append(fs_linear_reg_3(list(rSubset(cols,3,7)[r][i]), 'new_amount')[0])
        pred_l.append(fs_linear_reg_3(list(rSubset(cols,3,7)[r][i]), 'new_amount')[1])
dt = {'column_name': col_l, 'overall_mse': overall_l, 'pred_mse': pred_l}
f_table = pd.DataFrame.from_dict(dt)